<a href="https://colab.research.google.com/github/Choihankyul/deeplearing/blob/main/acupoint_deeplearning_model_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# kaggle Face Landmark Detection 참고

In [ ]:
import pandas as  pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras
import json
import cv2
import os

In [ ]:
# ## 데이터 가져오기
# %cd /content/drive/MyDrive/deeplearning_study/deeplearing
# !unzip -qq "/content/drive/MyDrive/deeplearning_study/deeplearing/img_json.zip"

In [ ]:
## img_json 파일에 있는 jpg,json 파일 이름 가져오기
csv_file = os.listdir('/content/drive/MyDrive/deeplearning_study/deeplearing/img_json')

# csv_file에서 json만 가져오기
json_list = []
for i in csv_file:
    file_form=i.split('.')[1]
    if file_form == 'json':
        json_list.append(i)

len(json_list)

In [ ]:
# json 파일로 불러오기
with open('/content/drive/MyDrive/deeplearning_study/deeplearing/img_json/100_json.json') as json_file:
  print(type(json_file))

In [ ]:
# 데이터 프레임으로 좌표값 저장
for j in json_list:
  with open(f'/content/drive/MyDrive/deeplearning_study/deeplearing/img_json/{j}','r' ,encoding='utf-8-sig') as json_file:
    json_data = json.load(json_file)
  if j == json_list[0]:
    df = pd.DataFrame(json_data).T
  else:
    new_df = pd.DataFrame(json_data).T
    df = pd.concat([df,new_df])  

In [ ]:
new_df = df.copy()

In [ ]:
## right 가 x 값이 작음

## X,Y 좌표로 전환

In [ ]:
# null 값이 없고 1개의 좌표만 있는 곳 : '인당','소료','태단'
list_nnc1 = ['인당','소료','태단']
onePoint_noNull_df = pd.DataFrame(index=new_df.index)
for column in list_nnc1:
  new_df[f'{column}_x'] = 0
  new_df[f'{column}_y'] = 0

# 빈리스트를 0,0 값으로 채우기
  def fill_zero(x):
    if not x:
      x.append([0,0])
    else:
      x
    return x
  new_df[f'{column}'] = new_df[f'{column}'].apply(lambda x:fill_zero(x))

# x,y 좌표 column 으로 저장
  for i in range(86):
    new_df[f'{column}_y'][i] = new_df[f'{column}'][i][0][1]
    new_df[f'{column}_x'][i] = new_df[f'{column}'][i][0][0]

# 새로운 데이터 프레임으로 저장
  onePoint_noNull_df[f'{column}_x'] = new_df[f'{column}_x']
  onePoint_noNull_df[f'{column}_y'] = new_df[f'{column}_y']

  new_df.drop([f'{column}',f'{column}_x',f'{column}_y'],axis=1,inplace=True)

In [ ]:
## 훈련을 방해할것 같은 ['81.jpg','115.jpg','117.jpg','127.jpg','32.jpg'] drop 하기
onePoint_noNull_df.drop(['81.jpg','115.jpg','117.jpg','127.jpg','32.jpg'],axis=0,inplace=True)

In [ ]:
# onePoint_noNull_df 확인
onePoint_noNull_df.info()

In [ ]:
# null 값이 있고 1개의 좌표만 있는 곳 : '수구' '승장'
list_hnullc1 = ['수구', '승장']
onePoint_haveNull_df = pd.DataFrame(index=new_df.index)
for column in list_hnullc1:
  new_df[f'{column}_x'] = 0
  new_df[f'{column}_y'] = 0

# 빈리스트를 0,0 값으로 채우기
  def fill_zero(x):
    if not x:
      x.append([0,0])
    else:
      x
    return x
  new_df[f'{column}'] = new_df[f'{column}'].apply(lambda x:fill_zero(x))

# x,y 좌표 column 으로 저장
  for i in range(86):
    new_df[f'{column}_y'][i] = new_df[f'{column}'][i][0][1]
    new_df[f'{column}_x'][i] = new_df[f'{column}'][i][0][0]

# 새로운 데이터 프레임으로 저장
  onePoint_haveNull_df[f'{column}_x'] = new_df[f'{column}_x']
  onePoint_haveNull_df[f'{column}_y'] = new_df[f'{column}_y']

  new_df.drop([f'{column}',f'{column}_x',f'{column}_y'],axis=1,inplace=True)

In [ ]:
## 훈련을 방해할것 같은 ['81.jpg','115.jpg','117.jpg','127.jpg','32.jpg'] drop 하기
onePoint_haveNull_df.drop(['81.jpg','115.jpg','117.jpg','127.jpg','32.jpg'],axis=0,inplace=True)

In [ ]:
# onePoint_haveNull_df 확인
onePoint_haveNull_df.info()

In [ ]:
# 2개의 좌표가 있고 null이 포함된 경혈점
list_hnullc2 = ['거료', '관료', '구후', '동자료', '사백', '사죽공', '상영향', '승읍', '양백', '어요', '영향','정명', '지창', '찬죽', '화료']
twoPoint_haveNull_df = pd.DataFrame(index=new_df.index)
for column in list_hnullc2:
  new_df[f'{column}_x1'] = 0
  new_df[f'{column}_y1'] = 0
  new_df[f'{column}_x2'] = 0
  new_df[f'{column}_y2'] = 0

# 빈리스트를 0,0 값으로 채우기
  def fill_zero(x):
    if not x:
      x.append([0,0])
      x.append([0,0])
    elif len(x) == 1:
      x.append([0,0])
    else:
      x
    return x
  new_df[f'{column}'] = new_df[f'{column}'].apply(lambda x:fill_zero(x))

# # x,y 좌표 column 으로 저장
  for i in range(86):
    if new_df[f'{column}'][i][0][0] != 0 and new_df[f'{column}'][i][1][0] != 0:
      
      if new_df[f'{column}'][i][0][0] > new_df[f'{column}'][i][1][0]:
        new_df[f'{column}_x1'][i] = new_df[f'{column}'][i][1][0] # x1 = 오른쪽(작은쪽)
        new_df[f'{column}_y1'][i] = new_df[f'{column}'][i][1][1]
        new_df[f'{column}_x2'][i] = new_df[f'{column}'][i][0][0] # x2 = 왼쪽(큰쪽)
        new_df[f'{column}_y2'][i] = new_df[f'{column}'][i][0][1]
      else:
        new_df[f'{column}_x1'][i] = new_df[f'{column}'][i][0][0]
        new_df[f'{column}_y1'][i] = new_df[f'{column}'][i][0][1]
        new_df[f'{column}_x2'][i] = new_df[f'{column}'][i][1][0]
        new_df[f'{column}_y2'][i] = new_df[f'{column}'][i][1][1]
    
    elif new_df[f'{column}'][i][0][0] == 0 and  new_df[f'{column}'][i][1][0] == 0 :
      new_df[f'{column}_x1'][i] = new_df[f'{column}'][i][0][0]
      new_df[f'{column}_y1'][i] = new_df[f'{column}'][i][0][1]
      new_df[f'{column}_x2'][i] = new_df[f'{column}'][i][1][0]
      new_df[f'{column}_y2'][i] = new_df[f'{column}'][i][1][1]

    else:
      if column in ['사죽공','관료','양백','어요','승읍','사백','거료','지창','찬죽','정명','상영향','영향']:
          if new_df[f'{column}'][i][0][0] >= 250:
                new_df[f'{column}_x1'][i] = new_df[f'{column}'][i][0][0] # 왼쪽
                new_df[f'{column}_y1'][i] = new_df[f'{column}'][i][0][1]
                new_df[f'{column}_x2'][i] = new_df[f'{column}'][i][1][0] # 오른쪽
                new_df[f'{column}_y2'][i] = new_df[f'{column}'][i][1][1]
          else:
                new_df[f'{column}_x1'][i] = new_df[f'{column}'][i][1][0]
                new_df[f'{column}_y1'][i] = new_df[f'{column}'][i][1][1]
                new_df[f'{column}_x2'][i] = new_df[f'{column}'][i][0][0]
                new_df[f'{column}_y2'][i] = new_df[f'{column}'][i][0][1]

      elif column in ['화료']:
          if new_df[f'{column}'][i][0][0] >= 255:
                new_df[f'{column}_x1'][i] = new_df[f'{column}'][i][0][0] # 왼쪽
                new_df[f'{column}_y1'][i] = new_df[f'{column}'][i][0][1]
                new_df[f'{column}_x2'][i] = new_df[f'{column}'][i][1][0] # 오른쪽
                new_df[f'{column}_y2'][i] = new_df[f'{column}'][i][1][1]
          else:
                new_df[f'{column}_x1'][i] = new_df[f'{column}'][i][1][0]
                new_df[f'{column}_y1'][i] = new_df[f'{column}'][i][1][1]
                new_df[f'{column}_x2'][i] = new_df[f'{column}'][i][0][0]
                new_df[f'{column}_y2'][i] = new_df[f'{column}'][i][0][1]


  twoPoint_haveNull_df[f'{column}_x1'] = new_df[f'{column}_x1']
  twoPoint_haveNull_df[f'{column}_y1'] = new_df[f'{column}_y1']
  twoPoint_haveNull_df[f'{column}_x2'] = new_df[f'{column}_x2']
  twoPoint_haveNull_df[f'{column}_y2'] = new_df[f'{column}_y2']
  
  new_df.drop([f'{column}',f'{column}_x1',f'{column}_y1',f'{column}_x2',f'{column}_y2'],axis=1,inplace=True)


In [ ]:
twoPoint_haveNull_df.info()

In [ ]:
## 훈련을 방해할것 같은 ['81.jpg','115.jpg','117.jpg','127.jpg','32.jpg'] drop 하기
twoPoint_haveNull_df.drop(['81.jpg','115.jpg','117.jpg','127.jpg','32.jpg'],axis=0,inplace=True)

In [ ]:
# twoPoint_haveNull_df 확인
twoPoint_haveNull_df.info()

In [ ]:
len(twoPoint_haveNull_df.index)

In [ ]:
## 데이터 프레임 합치기
All_df = pd.merge(twoPoint_haveNull_df,onePoint_haveNull_df,left_index=True,right_index=True,how='left')
All_df = pd.merge(All_df,onePoint_noNull_df,left_index=True,right_index=True,how='left')
All_df.head()

In [ ]:
# All_df 확인
All_df.info()

In [ ]:
# 저장
All_df.to_csv('/content/drive/MyDrive/deeplearning_study/data/acupoint_coordinate.csv',mode = 'w')